In [1]:
import pandas as pd
import os
from enum import Enum
import json
from tqdm import tqdm
import math
from pathlib import Path
import random
import shutil
import cv2
import numpy as np
import matplotlib.pyplot as plt

**SCARICA IL DATASET PER ALLENARE IL MODELLO YOLO**

In [ ]:
!wget -O data_for_YOLO_training.zip "https://www.dropbox.com/scl/fo/uwcu4mgrri2wukwndri0z/AIzxWBH9UTdXCLFuQXlKcX4?rlkey=4kgrbf2sxwlvxoqcwd62148fr&st=qoutrrqr&dl=0"
!unzip data_for_YOLO_training.zip

In [2]:
base_folder = os.path.join('.')

**SET PATH VARIABLES FOR TRAIN, TEST, AND VALIDATION FODLERS**

In [ ]:
data_yaml_path = os.path.join(base_folder, 'data.yaml')
source_folders = {
    'test': {
        'images': os.path.join(base_folder, 'splitted_cut_version', 'test', 'images'),
        'labels': os.path.join(base_folder, 'splitted_cut_version', 'test', 'labels')
    },
    'train': {
        'images': os.path.join(base_folder, 'splitted_cut_version', 'train', 'images'),
        'labels': os.path.join(base_folder, 'splitted_cut_version', 'train', 'labels')
    },
    'val': {
        'images': os.path.join(base_folder, 'splitted_cut_version', 'val', 'images'),
        'labels': os.path.join(base_folder, 'splitted_cut_version', 'val', 'labels')
    }
}

**CHECK THE FOLDERS TRAIN, TEST, AND VAL**

In [ ]:
total_images = 0
total_labels = 0
for split in ['train', 'val', 'test']:
    image_stem_list = sorted([Path(n).stem for n in os.listdir(
        source_folders[split]['images']) if n.endswith('.jpg') and not n.startswith('.')])
    label_stem_list = sorted([Path(n).stem for n in os.listdir(
        source_folders[split]['labels']) if n.endswith('.txt') and not n.startswith('.')])
    n_images = len(image_stem_list)
    n_labels = len(label_stem_list)
    total_labels += n_labels
    total_images += n_images
    print(f"La cartella {split}/images ha {n_images} elementi.")
    print(f"La cartella {split}/labels ha {n_labels} elementi.")
    for image_stem in image_stem_list:
        if image_stem not in label_stem_list:
            print(f"Image {image_stem}.jpg is not labelled.")
    for label_stem in label_stem_list:
        if label_stem not in image_stem_list:
            print(f"Image {label_stem}.txt has no image.")

print(f"Totale immagini: {total_images}")
print(f"Totale etichette: {total_labels}")

The folder train/images has 400 elements.
The folder train/labels has 400 elements.
The folder val/images has 405 elements.
The folder val/labels has 405 elements.
The folder test/images has 203 elements.
The folder test/labels has 203 elements.
Total images: 1008
Total labels: 1008


In [5]:
class Labels(Enum):
    gallina_bianca = 0
    gallina_rossa = 1

n_classes = 2
classes = ['gallina_bianca', 'gallina_rossa']

In [ ]:
!pip install ultralytics

In [ ]:
!pip install -U albumentations

In [ ]:
import torch
from ultralytics import YOLO

In [10]:
model = YOLO(os.path.join(base_folder, 'yolo11n.pt'))

In [ ]:
model.train(data=data_yaml_path, epochs=30)

In [ ]:
best_obj_detection_weights = os.path.join(base_folder, 'runs', 'detect', 'train', 'weights', 'best.pt')
model = YOLO(best_obj_detection_weights)
results = model.val()